In [66]:
import mlflow
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
from sklearn.feature_extraction import DictVectorizer

In [67]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.get_experiment_by_name(name="gamer")

<Experiment: artifact_location='mlflow-artifacts:/431548450445816159', creation_time=1718785153935, experiment_id='431548450445816159', last_update_time=1718785153935, lifecycle_stage='active', name='gamer', tags={}>

In [68]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='431548450445816159',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)
runs

[<Run: data=<RunData: metrics={'rmse': 5.242047147702308}, params={'learning_rate': '0.1212222356681564',
  'max_depth': '4',
  'min_child_weight': '3.830060509351922',
  'objective': 'reg:linear',
  'reg_alpha': '0.17455067131841767',
  'reg_lambda': '0.011875948364425093',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "d70b1b5572c942b59fcdd359ea982283", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2024-06-20 '
                              '06:41:13.161888", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.10.13", "data": '
                              '"model.xgb", "env": {"conda": "conda.yaml", '
                              '"virtualenv": "python_env.yaml"}}, "xgboost": '
                              '{"xgb_version": "2.0.3", "data": "model.xgb", '
                              '"mo

In [69]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d70b1b5572c942b59fcdd359ea982283, rmse: 5.2420
run id: 5892a1e8b8b4437bab15fe623a159e45, rmse: 5.2420
run id: fd113554612e4c45acd4377218db1271, rmse: 5.2420
run id: e67f10ff6da848c5bf454f6573f52078, rmse: 5.2420
run id: c11c47a3ff6b4843b344df66bf57b3ac, rmse: 5.2420


In [70]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [71]:
run_id = "d70b1b5572c942b59fcdd359ea982283"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="gamer")

Registered model 'gamer' already exists. Creating a new version of this model...
2024/06/20 09:52:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: gamer, version 5
Created version '5' of model 'gamer'.


<ModelVersion: aliases=[], creation_timestamp=1718877171644, current_stage='None', description='', last_updated_timestamp=1718877171644, name='gamer', run_id='d70b1b5572c942b59fcdd359ea982283', run_link='', source='mlflow-artifacts:/431548450445816159/d70b1b5572c942b59fcdd359ea982283/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

In [72]:
model_name = "gamer"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 5, stage: None


/tmp/ipykernel_2446/3018547499.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.14.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [73]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_2446/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.14.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1718868117792, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-06-20', last_updated_timestamp=1718877173446, name='gamer', run_id='d70b1b5572c942b59fcdd359ea982283', run_link='', source='mlflow-artifacts:/431548450445816159/d70b1b5572c942b59fcdd359ea982283/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [74]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1718868117792, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-06-20', last_updated_timestamp=1718877173570, name='gamer', run_id='d70b1b5572c942b59fcdd359ea982283', run_link='', source='mlflow-artifacts:/431548450445816159/d70b1b5572c942b59fcdd359ea982283/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [75]:
from sklearn.metrics import mean_squared_error
import pandas as pd
dv=DictVectorizer()

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.fit_transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}



In [76]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [77]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/codespaces-jupyter/Module-2/preprocessor'

In [78]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [79]:
X_test = preprocess(df, dv)

In [80]:
target = "duration"
y_test = df[target].values

In [82]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/home/codespace/.python/current/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.14.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/431548450445816159/d70b1b5572c942b59fcdd359ea982283/artifacts/model:
##### File  #####
API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/431548450445816159/d70b1b5572c942b59fcdd359ea982283/artifacts/model/ failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/431548450445816159/d70b1b5572c942b59fcdd359ea982283/artifacts/model/ (Caused by ResponseError('too many 500 error responses'))

In [83]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/home/codespace/.python/current/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.14.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


KeyboardInterrupt: 

In [85]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Staging",
    archive_existing_versions=True
)

/tmp/ipykernel_2446/712916771.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.14.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1718876938918, current_stage='Staging', description='', last_updated_timestamp=1718877499548, name='gamer', run_id='d70b1b5572c942b59fcdd359ea982283', run_link='', source='mlflow-artifacts:/431548450445816159/d70b1b5572c942b59fcdd359ea982283/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>